# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
Arxiv has 60 new papers today
          1 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/1 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2412.04557


extracting tarball to tmp_2412.04557...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2412.04557/paper.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'table_model.tex' from 'tmp_2412.04557/table_model.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'table_lines.tex' from 'tmp_2412.04557/table_lines.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-pack

Found 158 bibliographic references in tmp_2412.04557/paper.bbl.
Issues with the citations
syntax error in line 655: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.04557-b31b1b.svg)](https://arxiv.org/abs/2412.04557) | **An unambiguous AGN and a Balmer break in an Ultraluminous Little Red Dot at z=4.47 from Ultradeep UNCOVER and All the Little Things Spectroscopy**  |
|| I. Labbe, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-12-09*|
|*Comments*| *28 pages,10 figures, submitted to ApJ*|
|**Abstract**|            We present a detailed exploration of the most optically-luminous Little Red Dot ($L_{H\alpha}=10^{44}$erg/s, $L_V=10^{45}$erg/s, F444W=22AB) found to date. Located in the Abell 2744 field, source A744-45924 was observed by NIRSpec/PRISM with ultradeep spectroscopy reaching SNR$\sim$100pix$^{-1}$, high-resolution 3-4 micron NIRCam/Grism spectroscopy, and NIRCam Medium Band imaging. The NIRCam spectra reveal high rest-frame EW $W_{H\alpha,0,broad}>800$Å, broad H$\alpha$ emission (FWHM$\sim$4500 km/s), on top of narrow, complex absorption. NIRSpec data show exceptionally strong rest-frame UV to NIR Fe II emission ($W_{FeII-UV,0}\sim$340Å), N IV]$\lambda\lambda$1483,1486 and N III]$\lambda$1750, and broad NIR O I $\lambda$8446 emission. The spectra unambiguously demonstrate a broad-line region associated with an inferred $M_{BH}\sim10^9M_\odot$ supermassive black hole embedded in dense gas, which might explain a non-detection in ultradeep Chandra X-ray data (>$10\times$ underluminous relative to broad $L_{H\alpha}$). Strong UV Nitrogen lines suggest supersolar N/O ratios due to rapid star formation or intense radiation near the AGN. The continuum shows a clear Balmer break at rest-frame 3650Å, which cannot be accounted for by an AGN power-law alone. A stellar population model produces an excellent fit with a reddened Balmer break and implying a massive ($M_*\sim8\times10^{10}M_\odot$), old $\sim$500 Myr, compact stellar core, among the densest stellar systems known ($\rho\sim3\times10^6M_\odot$/pc$^2$ for $R_{e,opt}=70\pm10$ pc), and AGN emission with extreme intrinsic EW $W_{H\alpha,0}\gg$1000Å. However, although high $M_*$ and $M_{BH}$ are supported by evidence of an overdensity containing 40 galaxies at $z=4.41-4.51$, deep high-resolution spectroscopy is required to confirm stellar absorption and rule out that dense gas around the AGN causes the Balmer break instead.         |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2412.04557/./FIGS/stamps.png', 'tmp_2412.04557/./FIGS/spectrum_v1_msa4.png', 'tmp_2412.04557/./FIGS/stamps.png', 'tmp_2412.04557/./FIGS/spectrum_v1_msa4.png', 'tmp_2412.04557/./FIGS/agn_model.png']
copying  tmp_2412.04557/./FIGS/stamps.png to _build/html/
copying  tmp_2412.04557/./FIGS/spectrum_v1_msa4.png to _build/html/
copying  tmp_2412.04557/./FIGS/stamps.png to _build/html/
copying  tmp_2412.04557/./FIGS/spectrum_v1_msa4.png to _build/html/
copying  tmp_2412.04557/./FIGS/agn_model.png to _build/html/
exported in  _build/html/2412.04557.md
    + _build/html/tmp_2412.04557/./FIGS/stamps.png
    + _build/html/tmp_2412.04557/./FIGS/spectrum_v1_msa4.png
    + _build/html/tmp_2412.04557/./FIGS/stamps.png
    + _build/html/tmp_2412.04557/./FIGS/spectrum_v1_msa4.png
    + _build/html/tmp_2412.04557/./FIGS/agn_model.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\angstrom}{\text{ \normalfontÅ}\xspace}$
$\newcommand{\deltare}{\mathrm{\Delta log(r_e)}}$
$\newcommand{\oiii}{[\ion{O}{3}]\xspace}$
$\newcommand{\nii}{[\ion{N}{2}]\xspace}$
$\newcommand{\niilam}{[\ion{N}{2}]\lambda6549,6585\xspace}$
$\newcommand{\feii}{\ion{Fe}{2}\xspace}$
$\newcommand{\nev}{[\ion{Ne}{5}]\xspace}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\sersic}{Sérsic}$
$\newcommand{\mkgroup}{M_{\rm K,  group}}$
$\newcommand{\mstar}{M_*}$
$\newcommand{\mbh}{M_{\rm BH}}$
$\newcommand{\lbol}{L_{\rm bol}}$
$\newcommand{\jwst}{\ensuremath{JWST}}$
$\newcommand{\ivo}[1]{\textcolor{violet}{#1}}$
$\newcommand{\lt}{\textcolor{magenta!60!black}}$
$\newcommand{\catnr}{50,000}$
$\newcommand{\bronzenr}{40}$
$\newcommand{\silvernr}{26}$
$\newcommand{\goldnr}{17}$
$\newcommand{\sourcenum}{17\xspace}$
$\newcommand{\source}{A2744-45924\xspace}$
$\newcommand{\hide}[1]$
$\newcommand{\jw}{\emph{JWST}}$
$\newcommand{\angstrom}{\text{ \normalfontÅ}\xspace}$
$\newcommand{\deltare}{\mathrm{\Delta log(r_e)}}$
$\newcommand{\oiii}{[\ion{O}{3}]\xspace}$
$\newcommand{\nii}{[\ion{N}{2}]\xspace}$
$\newcommand{\niilam}{[\ion{N}{2}]\lambda6549,6585\xspace}$
$\newcommand{\feii}{\ion{Fe}{2}\xspace}$
$\newcommand{\nev}{[\ion{Ne}{5}]\xspace}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\sersic}{Sérsic}$
$\newcommand{\mkgroup}{M_{\rm K,  group}}$
$\newcommand{\mstar}{M_*}$
$\newcommand{\mbh}{M_{\rm BH}}$
$\newcommand{\lbol}{L_{\rm bol}}$
$\newcommand{\jwst}{\ensuremath{JWST}}$
$\newcommand{\ivo}[1]{\textcolor{violet}{#1}}$
$\newcommand{\lt}{\textcolor{magenta!60!black}}$
$\newcommand{\catnr}{50,000}$
$\newcommand{\bronzenr}{40}$
$\newcommand{\silvernr}{26}$
$\newcommand{\goldnr}{17}$
$\newcommand{\sourcenum}{17\xspace}$
$\newcommand{\source}{A2744-45924\xspace}$
$\newcommand{\hide}[1]$
$\newcommand{\jw}{\emph{JWST}}$
$\newcommand{\halpha}{\ensuremath{\mathrm{H}\alpha}\xspace}$
$\newcommand{\hbeta}{\ensuremath{\mathrm{H}\beta}\xspace}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\lyb}{Ly\beta}$
$\newcommand{\MgII}{Mg {\sc ii}\xspace}$
$\newcommand{\CIV}{C {\sc iv}\xspace}$
$\newcommand{\CIII}{C {\sc iii]}\xspace}$
$\newcommand{\FeII}{Fe {\sc ii}\xspace}$
$\newcommand{\NV}{N {\sc v}\xspace}$
$\newcommand{\HeII}{He {\sc ii}\xspace}$
$\newcommand{\MgIIwave}{Mg {\sc ii} \lambda2798}$
$\newcommand{\OIII}{[O {\sc iii}]}$
$\newcommand{\OII}{[O {\sc ii}]}$
$\newcommand{\NeIII}{[Ne {\sc iii}]}$
$\newcommand{\lgL}{\log \left({L_{\rm bol} \over{\rm erg s^{-1}}}\right)}$
$\newcommand{\halpha}{\ensuremath{\mathrm{H}\alpha}\xspace}$
$\newcommand{\hbeta}{\ensuremath{\mathrm{H}\beta}\xspace}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\lyb}{Ly\beta}$
$\newcommand{\MgII}{Mg {\sc ii}\xspace}$
$\newcommand{\CIV}{C {\sc iv}\xspace}$
$\newcommand{\CIII}{C {\sc iii]}\xspace}$
$\newcommand{\FeII}{Fe {\sc ii}\xspace}$
$\newcommand{\NV}{N {\sc v}\xspace}$
$\newcommand{\HeII}{He {\sc ii}\xspace}$
$\newcommand{\MgIIwave}{Mg {\sc ii} \lambda2798}$
$\newcommand{\OIII}{[O {\sc iii}]}$
$\newcommand{\OII}{[O {\sc ii}]}$
$\newcommand{\NeIII}{[Ne {\sc iii}]}$
$\newcommand{\lgL}{\log \left({L_{\rm bol} \over{\rm erg s^{-1}}}\right)}$</div>



<div id="title">

# An unambiguous AGN and a Balmer break in an Ultraluminous Little Red Dot at z=4.47 from Ultradeep UNCOVER and All the Little Things Spectroscopy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.04557-b31b1b.svg)](https://arxiv.org/abs/2412.04557)<mark>Appeared on: 2024-12-09</mark> -  _28 pages,10 figures, submitted to ApJ_

</div>
<div id="authors">

I. Labbe, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** We present a detailed exploration of the most optically-luminous Little Red Dot ( $L_{\mathrm{H}\alpha} = 10^{44}$ erg/s, $L_V=10^{45}$ erg/s, $\mathrm{F444W} = 22$ AB) discovered by JWST to date. Located in the $* UNCOVER*$ Abell 2744 field, source $\source$ was observed by NIRSpec/PRISM with ultradeep spectroscopy reaching $\langle \text{SNR} \rangle \sim 100   \text{pix}^{-1}$ , high-resolution $3-4$  $\micron$ NIRCam/Grism spectroscopy from the $* All the Little Things (ALT)*$ program, and multi-band NIRCam Medium Band imaging from $* Mega Science*$ . The NIRCam spectra reveal high rest-frame equivalent width $W_{\mathrm{H}\alpha,0,\mathrm{broad}} > 800$  $\angstrom$ , broad  H $\alpha$ emission ( $\mathrm{FWHM} \sim 4500$ km/s), on top of narrow, complex H $\alpha$ absorption. NIRSpec data show exceptionally strong rest-frame ultraviolet (UV) to near-infrared (NIR) $\ion{Fe}{2}$ emission ( $W_{\mathrm{FeII-UV},0} \sim 340$ Å), UV $\ion{N}{4}$ ]  $\lambda \lambda 1483, 1486$ and $\ion{N}{3}$ ]  $\lambda 1750$ , and broad NIR $\ion{O}{1}$ $\lambda8446$ emission. The line spectra unambiguously demonstrate a broad-line region (BLR) associated with a possible $M_{\mathrm{BH}} \sim 10^9 M_\odot$ supermassive black hole embedded in dense gas, which might explain the non-detection in ultradeep Chandra X-ray data ( $\gtrsim10\times$ underluminous relative to broad $L_{\halpha}$ ). Strong UV Nitrogen lines suggest supersolar N/O ratios due to rapid star formation or intense radiation fields near the AGN. The observed continuum shows a clear Balmer break at rest-frame $3650$ Å, which cannot be accounted for by power-law AGN alone. A stellar population model produces an excellent fit with a reddened Balmer break and would imply a massive ( $M_* \approx 8 \times 10^{10} M_{\odot}$ ), old $\sim 500$ Myr, compact stellar core, among the densest stellar systems known ( $\rho \approx 3 \times 10^6 M_{\odot}$ /pc $^2$ for $R_{e,\mathrm{optical}} = 70 \pm 10$ pc), and AGN emission lines with extreme intrinsic equivalent width $W_{\mathrm{H}\alpha,0} \gg 1000$  $\angstrom$ . However, although high $M_*$ and $M_{\mathrm{BH}}$ are supported by evidence of a galaxy overdensity containing 40 galaxies at $z=4.41-4.51$ , deep high-resolution spectroscopy is required to confirm stellar absorption and rule out that the Balmer break is instead caused by dense gas surrounding the AGN.

</div>

<div id="div_fig1">

<img src="tmp_2412.04557/./FIGS/stamps.png" alt="Fig19.1" width="50%"/><img src="tmp_2412.04557/./FIGS/spectrum_v1_msa4.png" alt="Fig19.2" width="50%"/>

**Figure 19. -** Top: broadband NIRCam images of $\source$ in the short wavelength channel filters F070W, F090W, F115W, F150W, F200W at 20 mas pixel size and long wavelength channel filters F277W, F356W, F444W at 40 mas pixel size. The images are $3$\arcsec$$ on a side. North is up, East is left. Also shown are color composite images and an overlay of the micro-shutter array locations of the NIRSpec PRISM observations. Middle: the 2D NIRSpec/PRISM spectrum used for analysis. Bottom: observed NIRSpec/PRISM spectrum (black) and error array (grey). Positions of key spectroscopic features are indicated. Top axis is rest-frame wavelength in micron. (*fig:thedata*)

</div>
<div id="div_fig2">

<img src="tmp_2412.04557/./FIGS/stamps.png" alt="Fig23.1" width="50%"/><img src="tmp_2412.04557/./FIGS/spectrum_v1_msa4.png" alt="Fig23.2" width="50%"/>

**Figure 23. -** Top: broadband NIRCam images of $\source$ in the short wavelength channel filters F070W, F090W, F115W, F150W, F200W at 20 mas pixel size and long wavelength channel filters F277W, F356W, F444W at 40 mas pixel size. The images are $3$\arcsec$$ on a side. North is up, East is left. Also shown are color composite images and an overlay of the micro-shutter array locations of the NIRSpec PRISM observations. Middle: the 2D NIRSpec/PRISM spectrum used for analysis. Bottom: observed NIRSpec/PRISM spectrum (black) and error array (grey). Positions of key spectroscopic features are indicated. Top axis is rest-frame wavelength in micron. (*fig:thedata*)

</div>
<div id="div_fig3">

<img src="tmp_2412.04557/./FIGS/agn_model.png" alt="Fig21" width="100%"/>

**Figure 21. -** Model fitting results using only AGN components (model I, top panels), only stellar components (model II, middle), and a mix of AGN and stars (model III, bottom). NIRSpec/PRISM observations (black), uncertainties (dotted), model fits (gray), and normalized residuals are shown. Model I consists of a dust-reddened AGN component (red) and a blue component (here parameterized as a fraction of dust-free scattered light; blue), which dominates at $<2 \micron$ despite being only a small percentage of the red luminosity. The model can produce an apparent sharp break at $\sim3500$Å due to a blend of strong broad emission lines and broad $\ion${Fe}{2} pseudo-continuum at $4000-5500Å$, but leaves significant residuals throughout the rest-frame optical. Model II consists of two FSPS population components, each velocity-broadened according to the stellar mass and effective radius at F300W and where the broad lines are stellar in origin. AGN specific lines are masked. Model III is a mix of AGN and a stellar component. This fit prefers an evolved stellar population to dominating the rest-frame optical to match the observed Balmer break, with significant contribution from AGN emission lines and $\ion${Fe}{2}. Between these models, model III is favored by the data.
 (*fig:agn_detailed_fit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.04557"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

271  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
